Compare N-mod SVD and SVD

Compare different Identities and Expression

In [7]:
import scipy.io as sio
import struct
import numpy as np
import os
from sklearn.decomposition import PCA
import openmesh

n_bs = 0
n_vert = 0
n_face = 0
n_people = 150
path_prefix = '/home/jingwang/Data/data/FaceWarehouse/'
result_path = '/home/jingwang/Data/data/FaceWarehouse/preprocessed/'
n_training_pose = 20

n_test = 10
n_train = n_people - n_test


def load_bs_file(bs_path):
    global n_bs, n_vert, n_face
    with open(bs_path, 'rb') as fin:
        n_bs, n_vert, n_face = struct.unpack('iii', fin.read(4 * 3))
        # 46 11510 11400
        neu_vert = np.fromfile(fin, dtype=np.float32,
                               count=(n_bs + 1) * n_vert * 3)
        return neu_vert


def load_training_pose(idx):
    ten = []
    template = os.path.join(path_prefix, 'Tester_{}'.format(
        idx), 'TrainingPose', 'pose_aligned_{}.obj')
    for i in range(n_training_pose):
        file_path = template.format(i)
        mesh = openmesh.read_trimesh(file_path)
        ten.append(mesh.points())
    ten = np.array(ten,dtype=np.float32)
    return ten


def load_personalized_blendshape(idx):
    file_path = os.path.join(
        path_prefix, 'Tester_{}'.format(idx), 'Blendshape', 'shape.bs')
    tmp = load_bs_file(file_path)
    tmp = tmp.reshape(n_bs+1, n_vert, 3)
    return tmp

In [2]:
ten_train = []
ten_test = []
n_exp_test = 5
for i in range(1, n_people + 1):
    blendshape = load_personalized_blendshape(i) # 47 x 11510 x 3
    ten_train.append(blendshape[:-n_exp_test])
    ten_test.append(blendshape[-n_exp_test:])
    
ten_test = np.concatenate(ten_test, axis=0)
ten_train = np.concatenate(ten_train, axis=0)
print(ten_test.shape, ten_train.shape)
np.save(os.path.join(result_path, 'train_last_{}_exp.npy'.format(n_exp_test)), ten_train)
np.save(os.path.join(result_path, 'test_last_{}_exp.npy'.format(n_exp_test)), ten_test)

(750, 11510, 3) (6300, 11510, 3)


In [3]:
6300/150

42.0

exp10: All exp, test on new identities

In [5]:
from sklearn.decomposition import PCA

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
pca = PCA(n_components=args['nz'])
X = np.array([dataset_train[i]['points'].cpu().numpy() for i in range(len(dataset_train))])
X = X.reshape(X.shape[0], -1)
W = pca.fit_transform(X.T)
X_val = np.array([dataset_test[i]['points'].cpu().numpy() for i in range(len(dataset_test))]) # prevbug forget to modify in len()
X_mean = np.mean(X, axis=0)
X_val = X_val.reshape(X_val.shape[0], -1).T
x = np.linalg.lstsq(W,X_val-X_mean.reshape(-1,1),rcond=None)[0]
pca_recon = np.matmul(W,x)+X_mean.reshape(-1,1)
X_val = X_val[:-3]
pca_recon = pca_recon[:-3]
shapedata.save_meshes(os.path.join(worst_mesh_path,'pca'),pca_recon.T, list(range(pca_recon.shape[0])))
shapedata.save_meshes(os.path.join(worst_mesh_path,'pca_input'),X_val.T, list(range(X_val.shape[0])))
X_val_denorm = X_val * shapedata.std.reshape(-1,1) + shapedata.mean.reshape(-1,1)
pca_denorm = pca_recon * shapedata.std.reshape(-1,1) + shapedata.mean.reshape(-1,1)
residual = (X_val_denorm-pca_denorm)**2
residual = residual.reshape(-1,3,10)
residual = np.sqrt(np.sum(residual,axis=1))
np.mean(residual)*100

In [139]:
import pathlib
template_path = '/home/jingwang/Data/data/FaceWarehouse/template/template.obj'
def gen_separate_exp(n_id,last_exp):
    for cur_id in range(1,n_id+1):
        for cur_exp in range(-last_exp,0):
            yield 'id_{0}_exp_{1}'.format(cur_id, cur_exp)
def gen_sep_id(n_id, n_exp):
    for cur_id in range(1,n_id+1):
        for cur_exp in range(n_exp):
            yield 'id_{}_exp_{}'.format(cur_id, cur_exp)

In [ ]:
def test_pca_wrong(train_path, test_path, mesh_output_path, nz, template_path, name_gen, should_save_mesh=False):
    train_data = np.load(train_path)
    mean = np.mean(train_data, axis=0).reshape(1, -1, 3)
    train_data -= mean
    mean = mean.reshape(1, -1)
    test_data = np.load(test_path)
    pca = PCA(n_components=nz)
    train_data = train_data.reshape(train_data.shape[0], -1)
    test_data = test_data.reshape(test_data.shape[0], -1)
    W = pca.fit_transform(train_data.T)
    x = np.linalg.lstsq(W, (test_data-mean).T, rcond=None)
    recon = W@x[0]+mean.T
    recon = recon.T
    recon = recon.reshape(recon.shape[0], -1, 3)
    test_data = test_data.reshape(test_data.shape[0], -1, 3)
    template = openmesh.read_trimesh(template_path)
    pathlib.Path(mesh_output_path).mkdir(exist_ok=True, parents=True)
    if should_save_mesh:
        for i, name in enumerate(name_gen):
            save_mesh(test_data[i], template, os.path.join(
                mesh_output_path, 'test_'+name+'.obj'))
            save_mesh(recon[i], template, os.path.join(
                mesh_output_path, 'recon_'+name+'.obj'))
    residual = np.sqrt(np.sum((test_data - recon)**2, axis=2))
    return np.mean(residual)*100  # convert dm to mm

In [266]:
def save_mesh(points, template, output_path):
    template.points()[:] = points
    print('saving', output_path)
    openmesh.write_mesh(output_path, template)

def test_pca(train_path, test_path, mesh_output_path, nz, template_path, name_gen, should_save_mesh=False):
    train_data = np.load(train_path)
    test_data = np.load(test_path)
    pca = PCA(n_components=nz)
    train_data = train_data.reshape(train_data.shape[0], -1)
    test_data = test_data.reshape(test_data.shape[0], -1)
    pca.fit(train_data)
    recon = pca.inverse_transform(pca.transform(test_data))
    recon = recon.reshape(recon.shape[0], -1, 3)
    test_data = test_data.reshape(test_data.shape[0], -1, 3)
    template = openmesh.read_trimesh(template_path)
    pathlib.Path(mesh_output_path).mkdir(exist_ok=True, parents=True)
    if should_save_mesh:
        print('start saving')
        for i, name in enumerate(name_gen):
            save_mesh(test_data[i], template, os.path.join(
                mesh_output_path, 'test_'+name+'.obj'))
            save_mesh(recon[i], template, os.path.join(
                mesh_output_path, 'recon_'+name+'.obj'))
    residual = np.sqrt(np.sum((test_data - recon)**2, axis=2))
    return np.mean(residual)*100  # convert dm to mm

In [144]:
test_pca('/home/jingwang/Data/data/FaceWarehouse/preprocessed/train_last_5_exp.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/train_last_5_exp.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_sep_exp',
        128,
        template_path,
        gen_separate_exp(150,5),
        should_save_mesh=False)

0.1778074773028493

Up, test on train set

In [142]:
test_pca('/home/jingwang/Data/data/FaceWarehouse/preprocessed/train_last_5_exp.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/test_last_5_exp.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_sep_exp',
        128,
        template_path,
        gen_separate_exp(150,5),
        should_save_mesh=True)

0.3116756211966276

Up, generalize on new expressions

In [143]:
test_pca('/home/jingwang/Data/data/FaceWarehouse/preprocessed/align_pose/train_bs_only.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/align_pose/test_bs_only.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_sep_id',
        128,
        template_path,
        gen_sep_id(10,47),
        should_save_mesh=True)

0.470117898657918

In [147]:
test_pca('/home/jingwang/Data/data/FaceWarehouse/preprocessed/align_pose/train_bs_only.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/align_pose/train_bs_only.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_sep_id',
        128,
        template_path,
        gen_sep_id(10,47),
        should_save_mesh=False)

0.17254167469218373

Up, generalize on new unseen id

In [191]:
tmp = np.load('/home/jingwang/Data/data/FaceWarehouse/preprocessed/neutral/train.npy')

In [192]:
tmp.shape

(150, 11510, 3)

In [216]:
np.save('/home/jingwang/Data/data/FaceWarehouse/preprocessed/neutral/test_last_10.npy',tmp[140:])

In [217]:
np.save('/home/jingwang/Data/data/FaceWarehouse/preprocessed/neutral/train_first_140.npy',tmp[:140])

In [221]:
test_pca('/home/jingwang/Data/data/FaceWarehouse/preprocessed/neutral/train_first_140.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/neutral/test_last_10.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral',
        16,
        template_path,
        gen_sep_id(10,1),
        should_save_mesh=True)

start saving
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/test_id_1_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/recon_id_1_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/test_id_2_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/recon_id_2_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/test_id_3_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/recon_id_3_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/test_id_4_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/recon_id_4_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/test_id_5_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/recon_id_5_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/t

1.4661343768239021

In [220]:
test_pca('/home/jingwang/Data/data/FaceWarehouse/preprocessed/neutral/train_first_140.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/neutral/train_first_140.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral',
        16,
        template_path,
        gen_sep_id(10,1),
        should_save_mesh=False)

1.1846146546304226

In [215]:
test_pca('/home/jingwang/Data/data/FaceWarehouse/preprocessed/neutral/train_140.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/neutral/test_10.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral',
        16,
        template_path,
        gen_sep_id(10,1),
        should_save_mesh=True)

start saving
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/test_id_1_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/recon_id_1_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/test_id_2_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/recon_id_2_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/test_id_3_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/recon_id_3_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/test_id_4_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/recon_id_4_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/test_id_5_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/recon_id_5_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral/t

1.6180649399757385

In [214]:
test_pca('/home/jingwang/Data/data/FaceWarehouse/preprocessed/neutral/train_140.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/neutral/train_140.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/pca_neutral',
        16,
        template_path,
        gen_sep_id(140,1),
        should_save_mesh=False)

1.1815941892564297

In [ ]:
def test_bilinear(train_path, test_path, mesh_output_path, nz_id, nz_exp, n_id, n_exp,
                  template_path, name_gen, should_save_mesh=False):
    train_data = np.load(train_path)
    test_data = np.load(test_path)
    test_data = test_data[:10] # fixme, just because it's slow
    pca_id = PCA(n_components=nz_id)
    pca_exp = PCA(n_components=nz_exp)
    test_data = test_data.reshape(test_data.shape[0], -1)
    train_data = train_data.reshape(n_id, -1)
    pca_id.fit(train_data)
    # (nz_id+1) x n_exp*n_ver*3
    exp_data = np.vstack([pca_id.mean_, pca_id.components_])
    exp_data = exp_data.reshape(
        nz_id+1, n_exp, -1).transpose([1, 0, 2]).reshape(n_exp, -1)
    # prevbug: nz_id should +1
    pca_exp.fit(exp_data)
    core_tensor = np.vstack([pca_exp.mean_, pca_exp.components_]).reshape(
        nz_exp+1, nz_id+1, -1)
    print('computed core tensor')
    recon = []
    
    pathlib.Path(mesh_output_path).mkdir(exist_ok=True, parents=True)
    template = openmesh.read_trimesh(template_path)
    
    for i in range(test_data.shape[0]):
        id_x = np.zeros(nz_id)
        exp_x = np.zeros(nz_exp)
        n_iter = 12
        for iter_i in range(n_iter):
            bs = np.tensordot(core_tensor, np.concatenate(
                [[1], id_x]), axes=[[1], [0]])
            exp_x = np.linalg.lstsq(bs[1:].T, test_data[i]-bs[0], rcond=None)[0]
            idb = np.tensordot(core_tensor, np.concatenate(
                [[1], exp_x]), axes=[[0], [0]])
            id_x = np.linalg.lstsq(idb[1:].T, test_data[i]-idb[0], rcond=None)[0]

        recon.append(np.tensordot(
            np.tensordot(core_tensor,
                         np.concatenate([[1], exp_x]),
                         axes=[[0], [0]]),
            np.concatenate([[1], id_x]),
            axes=[[0], [0]]))
        print('done {}'.format(i))
        if should_save_mesh:
            name = next(name_gen)
            save_mesh(test_data[i].reshape(-1, 3), template, os.path.join(
                mesh_output_path, 'test_'+name+'.obj'))
            save_mesh(recon[i].reshape(-1, 3), template, os.path.join(
                mesh_output_path, 'recon_'+name+'.obj'))
        print('saved {}'.format(i))
                
    recon = np.array(recon)
    test_data = test_data.reshape(test_data.shape[0], -1, 3)
    recon = recon.reshape(recon.shape[0], -1, 3)

    residual = np.sqrt(np.sum((test_data - recon)**2, axis=2))
    return np.mean(residual)*100  # convert dm to mm

In [269]:
core_tensor.shape

(29, 101, 34530)

In [270]:
id_x = np.concatenate([[1],np.zeros(100)])

In [271]:
id_x.shape

(101,)

In [272]:
tmp = np.tensordot(core_tensor, id_x, axes=[[1],[0]])

In [273]:
tmp.shape

(29, 34530)

In [274]:
nz_exp

28

In [275]:
tmp[1:].shape

(28, 34530)

In [264]:
test_data.shape

(470, 34530)

In [298]:
test_bilinear('/home/jingwang/Data/data/FaceWarehouse/preprocessed/align_pose/train_bs_only.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/align_pose/test_bs_only.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/bi_sep_id',
        100,28,140,47,
        template_path,
        gen_sep_id(10,47),
        should_save_mesh=True)

computed core tensor
done 0
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/bi_sep_id/test_id_1_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/bi_sep_id/recon_id_1_exp_0.obj
saving /home/jingwang/Data/data/FaceWarehouse/preprocessed/bi_sep_id/test_id_1_exp_1.obj


IndexError: list index out of range

In [ ]:
test_bilinear('/home/jingwang/Data/data/FaceWarehouse/preprocessed/align_pose/train_bs_only.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/align_pose/test_bs_only.npy',
        '/home/jingwang/Data/data/FaceWarehouse/preprocessed/bi_sep_id_88_40',
        88,40,140,47,
        template_path,
        gen_sep_id(10,47),
        should_save_mesh=True)

In [290]:
test_data.reshape(test_datast)

(10, 34530)

In [245]:
exp_data.shape

(47, 3453000)

In [246]:
pca_exp.mean_.shape

(3453000,)

In [263]:
core_tensor.shape

(29, 101, 34530)

In [235]:
exp_data.shape

(47, 3453000)

In [236]:
pca_id.components_.shape

(100, 1622910)

In [250]:
pca_exp.components_.shape

(28, 3453000)

In [156]:
pca_id.mean_.shape

(1622910,)

In [157]:
pca_exp.mean_.shape

(3453000,)

In [158]:
core_tensor.shape

(28, 100, 34530)

In [162]:
47*34530

1622910

In [161]:
100*34530

3453000

In [159]:
exp_data.shape

(47, 3453000)

In [126]:
train_data.shape

(750, 34530)

In [132]:
pca.mean_.shape

(34530,)

In [127]:
pca = PCA(150)

In [128]:
pca.fit(train_data)

PCA(copy=True, iterated_power='auto', n_components=150, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [129]:
pca.components_.shape

(150, 34530)

In [131]:
pca.inverse_transform(pca.transform(train_data)).shape

(750, 34530)

In [63]:
W.shape

(34530, 750)

In [90]:
np.dot(W[:,0],W[:,1])

0.0019378662

In [91]:
np.dot(W[0],W[1])

-0.073478766

In [93]:
np.dot(W[1],W[2])

-0.04603089

In [122]:
pca.components_.shape

(600, 750)

In [124]:
np.mean([np.dot(pca.components_[i],pca.components_[j]) for i in range(600) for j in range(600) if i!=j])

-2.0303174e-10

In [125]:
np.mean([np.dot(pca.components_[:,i],pca.components_[:,j]) for i in range(600) for j in range(600) if i!=j])

-0.0010339294

In [94]:
np.dot(W[:,1],W[:,2])

-0.00031280518

In [64]:
tmp.shape

(34530,)

In [186]:
tmp_data = np.array([
    [1,2,3],
    [3,4,10],
    [6,7,8]
])

In [187]:
tmp_pca = PCA(3)

In [188]:
tmp_pca.fit_transform(tmp_data)

array([[ 5.16676770e+00, -4.39773150e-01,  4.32689795e-16],
       [-1.81417842e+00,  2.43536011e+00,  4.32689795e-16],
       [-3.35258928e+00, -1.99558696e+00,  4.32689795e-16]])

In [189]:
tmp_pca.components_

array([[-4.94615366e-01, -4.94615366e-01, -7.14640665e-01],
       [-5.05327261e-01, -5.05327261e-01,  6.99491758e-01],
       [-7.07106781e-01,  7.07106781e-01, -1.38953306e-16]])

In [185]:
tmp_pca.explained_variance_ratio_

array([1.00000000e+00, 2.81351049e-34])

In [184]:
np.linalg.det(tmp_pca.components_)

-1.0

In [60]:
w = np.array([[1,2]])
print(w.shape)
np.linalg.pinv(w).shape

(1, 2)


(2, 1)

In [169]:
pca1 = PCA(100)
pca2 = PCA(28)

In [174]:
train_data_id = train_data.reshape(150,42,34530)

In [175]:
W1 = pca1.fit_transform(train_data.reshape(150,-1).T).T

In [176]:
W1 = W1.reshape(100,42,34530)

In [179]:
W2 = pca2.fit_transform(W1.transpose([1,0,2]).reshape(42,-1).T).T

In [180]:
W2 = W2.reshape(28,100,11510,3)

In [181]:
train_data_sep = train_data.reshape(150,42,34530)

In [182]:
mean_blendshape = np.mean(train_data_sep, axis=0)

In [192]:
test_data = ten_test.reshape(750,34530)

In [184]:
W2 = W2.reshape(28,100,34530)

In [187]:
for i in range(test_data.shape[0]):
    target = test_data[i]
    id_x = np.zeros(100)
    exp_x = np.zeros(28)
    for it in range(3):
        bs = np.tensordot(W2,id_x,)

LinAlgError: 3-dimensional array given. Array must be two-dimensional

In [200]:
pca.mean_.shape

(6300,)

In [102]:
DMM_path = '/home/jingwang/Data/git-task/opencv-face/data/BFM.mat'

In [96]:
import scipy.io as sio

In [103]:
bfm = sio.loadmat(DMM_path)

In [104]:
bfm.keys()

dict_keys(['__header__', '__version__', '__globals__', 'model'])

In [112]:
bfm['model']['shapeMU'].item().shape

(159645, 1)

In [114]:
shapePC = bfm['model']['shapePC'].item()

In [118]:
np.mean([np.dot(shapePC[:,i],shapePC[:,j]) for i in range(199) for j in range(199) if i!=j])

3.3273085e-05

In [117]:
np.sum([np.dot(shapePC[i,:],shapePC[j,:]) for i in range(199) for j in range(199) if i!=j])

3.7291107